In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-23 23:17:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2021-04-23 23:17:09 (6.02 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
#Create DataFrames to match tables
from pyspark.sql.functions import to_date
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Read in the Review dataset as a DataFrame

In [6]:
# Create the customers_table DataFrame
#customers_df = df.groupby("customer_id").agg({""})
customers_df = df.groupby('customer_id').count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45657423|             1|
|   47663761|             1|
|   11931196|             1|
|   44620216|             1|
|    1167081|             1|
|   17682867|             1|
|   10782454|             1|
|   51881951|             1|
|    1537684|             1|
|   17133428|             5|
|     661760|             1|
|    9867391|             9|
|   31268594|             1|
|   48581590|             1|
|   32466409|             3|
|   38490515|             1|
|   41318181|             4|
|   18800155|             6|
|   46306239|             1|
|   28409312|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LHJMHVS|FAIRY COUPLE Wome...|
|B00TPWQ6I4|Chic Alert Medica...|
|B00P837QR4|The Original Roll...|
|B00L280CN4|10pcs 18" 18 Inch...|
|B00KGU5T4Q|KONOV Mens Stainl...|
|B00JHF388K|Sterling Silver w...|
|B00FB05CES|CHOOSE YOUR COLOR...|
|B00D4KUMQ0|INBLUE Men's Stai...|
|B00GTCD2X0|Beautiful Carved ...|
|B00QUHZQP6|Authentic Chamili...|
|B00NJ317T8|Smitco LLC Silver...|
|B00658M8EC|925 Sterling Silv...|
|B00L43PV2Y|Hydia Steel Jewel...|
|B00U24CSP4|Platinum-Plated S...|
|B006OHTHZ2|316L Stainless St...|
|B00QW72KDA|PiercingJ Pair 12...|
|B006VN35L6|5mm Stainless Ste...|
|B00CGID7PY|925 Sterling Silv...|
|B00TXHSXWI|Silver Forest Kno...|
|B004Z1EQRS|PammyJ Silvertone...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGP1I51Y9LY5I|   46167946|B009SF48VU|     969375548| 2015-08-31|
|R2188MGKCZCZ0S|   40311851|B00DCXPY1M|     611202721| 2015-08-31|
|R3V5K1M153FV1T|   50584503|B00J3VBC5E|     887517157| 2015-08-31|
|R3S9D3NKL17SHR|   49335076|B00M4IEPKC|     215128937| 2015-08-31|
|R263VNMAVORV7L|   52877710|B00C15YRWO|      48773140| 2015-08-31|
|R1MXN8HYNSHLW7|   10916547|B00GEVB6PM|     541440034| 2015-08-31|
| RACHMLUWGE02P|   29016815|B00JF004Y8|     357701237| 2015-08-31|
|R20LF4RBJ34U4I|   18440567|B0035LBXXY|      13582621| 2015-08-31|
|R35QHWMJ1XI773|   47074101|B014GC4JMM|      42531962| 2015-08-31|
|R21WLOIEQL00VB|   11434871|B0085CNRK0|     858011593| 2015-08-30|
|R3EE0TJH1EP282|    5553301|B00SKPW9XS|     684124909| 2015-08-30|
|R2JE40P9QLDWCE|    1356449|B011J9YCAE|      64860477| 2015-08

In [9]:
#Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df = vine_df.select(F.col('review_id'),F.col('star_rating').cast('int'),F.col('helpful_votes').cast('int'),F.col('total_votes').cast('int'),F.col('vine'),F.col('verified_purchase'))
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [10]:
#deliverable 2

In [31]:
vine_df2 = vine_df.select(F.col('review_id'),F.col('star_rating').cast('float'),F.col('helpful_votes').cast('float'),F.col('total_votes').cast('float'),F.col('vine'),F.col('verified_purchase'))
vine_df2 = vine_df2.filter(df.total_votes >= 20)
vine_df2 = vine_df2.filter(F.col('helpful_votes')/F.col('total_votes')>=0.5)

vine_df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|        5.0|         23.0|       23.0|   N|                Y|
| R5O4WE9UM60B0|        1.0|         29.0|       29.0|   N|                Y|
|R2EKRVCRC7U0IY|        1.0|         22.0|       24.0|   N|                Y|
|R2OKV47GETH0L7|        5.0|         24.0|       24.0|   N|                Y|
| R36LII9IITE17|        1.0|         22.0|       24.0|   N|                Y|
|R35VKNE16PFY0H|        5.0|         22.0|       24.0|   N|                N|
|R3TG008LHHZZIN|        5.0|        123.0|      125.0|   N|                Y|
|R2FYQBKCC1XG4M|        5.0|         20.0|       20.0|   N|                Y|
| R63PV336NI5X7|        1.0|         27.0|       28.0|   N|                Y|
|R1R42WPHB5ZSWI|        5.0|         51.0|       52.0|   N|     

In [37]:
# Number of Paid Reviews
vine_df2Y = vine_df2.filter(df.vine == 'Y')
vine_df2Y.count()

21

In [38]:
# Number of Unpaid Reviews
vine_df2N = vine_df2.filter(df.vine == 'N')
vine_df2N.count()

7689

In [39]:
# Total Number of helpful Reviews  
vine_df2.count()

7710

In [41]:
# Total Number of helpful 5 star reviews
vine_df2.filter(df.star_rating==5.0).count()

4455

In [44]:
# Percentage of 5 star reviews (paid)
vine_df2Y.filter(df.star_rating==5.0).count()/vine_df2Y.count()

0.5238095238095238

In [45]:
# Percentage of 5 star reviews (unpaid)
vine_df2N.filter(df.star_rating==5.0).count()/vine_df2N.count()

0.5779685264663805